# Project 1: Digital Divide
### Data Prep

#### Based on PPIC's Just the Facts report ["California's Digital Divide"](https://www.ppic.org/publication/californias-digital-divide/)

## Research Question(s):
1. What share households in X state have access to high-speed internet? 
2. Does this number vary across demographic groups? (in this case race/ethnicity).

## Goal:
* explore datafiles (`acsdata.data.gz`) and create a _working dataset_ from it.

## Context:
Obtained American Community Survey (ACS) survey data from [IPUMS](https://usa.ipums.org/usa/). <br>
It contains basic demographics:
  - age
  - gender
  - race/ethnicity

and geographic indicators:
  - state
  - county

***

#### Step 1: Set up your working environment.

Import all necessary libraries and create `Path`s to your data directories. This ensures reproducibility across file systems (windows uses `\` instead of `/`)

We need 
1. `pandas` to work with the data.
2. `pathlib`, and more specifically its `Path` object, to work with paths. This will ensure our code works in both Windows (which uses `\` in its file paths) and MacOS/Linux (which uses `/`).
3. `datetime` - tip: There are version control systems for data but tagging your data files with the date is not a bad first step if you're getting started.

In [1]:
# setting up working environment
import pandas as pd
from pathlib import Path
from datetime import datetime as dt
today = dt.today().strftime("%d-%b-%y")

print(today)

01-May-19


In [2]:
# data folder and paths
RAW_DATA_PATH = Path("../data/raw/")
INTERIM_DATA_PATH = Path("../data/interim/")
PROCESSED_DATA_PATH = Path("../data/processed/")
FINAL_DATA_PATH = Path("../data/final/")

**NOTE:** I've included a `tools.py` script with the function `tree` which displays a directory's tree (obtained from [RealPython's tutorial on the `pathlib` module](https://realpython.com/python-pathlib/)).

    from our tools script import tree so we can use it.

In [3]:
from tools import tree

In [4]:
tree(RAW_DATA_PATH)

+ ../data/raw
    + .DS_Store
    + acs_data.csv.gz
    + acs_data.dta.gz
    + cps_data.dta.gz


***

#### Step 2: Load and explore the data

With `pandas` loading data is as easy as `.read_csv(PATH_TO_CSV_FILE)` and that works most of the time. `Pandas` `read_csv()` is so powerful it'll even read compressed files without any other parameter specification. Try the following:

```python
data = pd.read_csv(RAW_DATA_PATH / 'acs_data.csv.gz')
data.head()
```
_*make sure you change_ `RAW_DATA_PATH` _to match whatever variable name you chose for it earlier._

In [5]:
data = pd.read_csv(RAW_DATA_PATH / 'acs_data.csv.gz')
data.head()

,YEAR,SERIAL,HHWT,STATEFIP,COUNTYFIP,GQ,CINETHH,CIHISPEED,PERNUM,PERWT,RELATE,RELATED,SEX,AGE,RACE,RACED,HISPAN,HISPAND
0,2017,1,206,1,0,1,3,0,1,206,1,101,1,73,2,200,0,0
1,2017,2,45,1,0,1,1,20,1,45,1,101,2,31,1,100,0,0
2,2017,3,136,1,0,1,1,20,1,136,1,101,1,41,1,100,1,100
3,2017,3,136,1,0,1,1,20,2,121,2,201,2,48,1,100,0,0
4,2017,3,136,1,0,1,1,20,3,111,3,301,1,16,1,100,1,100


***
IPUMS offers a few data formats which can be more useful [[docs]](https://usa.ipums.org/usa-action/faq#ques12):
> In addition to the ASCII data file, the system creates a statistical package syntax file to accompany each extract. The syntax file is designed to read in the ASCII data while applying appropriate variable and value labels. SPSS, SAS, and Stata are supported. You must download the syntax file with the extract or you will be unable to read the data. The syntax file requires minor editing to identify the location of the data file on your local computer.

In this case, we'll be using a **Stata** file (`.dta`). The main reason is that `.dta` files can store *value labels* which `pandas` can then read and convert columns to `Categorical` columns in our pandas DataFrame. This 1) saves memory, and 2) is good practice because certain social sciences really, _really_, ***really*** love Stata so their interesting datasets are likely `.dta` files. 

However, `pandas` cannot read compressed `.dta` directly like it can `.csv` files. IPUMS, uses *gzip* compressed format and `python` includes a `gzip` module in its standard library.

**Import** gzip and try the following:
```python
with gzip.open(RAW_DATA_PATH / 'acs_data.dta.gz') as file:
    data = pd.read_stata(file)
```

and then display the first five rows of your `data` DataFrame.

In [6]:
# import gzip and load data
import gzip
with gzip.open(RAW_DATA_PATH / 'acs_data.dta.gz') as file:
    data = pd.read_stata(file)

In [7]:
# display first 5 rows
data.head()

,year,serial,hhwt,statefip,countyfip,gq,cinethh,cihispeed,pernum,perwt,relate,related,sex,age,race,raced,hispan,hispand
0,2017,1,206,alabama,0,households under 1970 definition,"no internet access at this house, apartment, o...",n/a (gq),1,206,head/householder,head/householder,male,73,black/african american/negro,black/african american/negro,not hispanic,not hispanic
1,2017,2,45,alabama,0,households under 1970 definition,"yes, with a subscription to an internet service",no,1,45,head/householder,head/householder,female,31,white,white,not hispanic,not hispanic
2,2017,3,136,alabama,0,households under 1970 definition,"yes, with a subscription to an internet service",no,1,136,head/householder,head/householder,male,41,white,white,mexican,mexican
3,2017,3,136,alabama,0,households under 1970 definition,"yes, with a subscription to an internet service",no,2,121,spouse,spouse,female,48,white,white,not hispanic,not hispanic
4,2017,3,136,alabama,0,households under 1970 definition,"yes, with a subscription to an internet service",no,3,111,child,child,male,16,white,white,mexican,mexican


***

#### Step 3: Familiarize yourself with the dataset

We've already seen `.head()` - the `pandas` method that will display the first 5 rows of your DataFrame. This gives you an idea of what your data looks like. However, there are is a lot more `.info()` you can get out of your dataframe. You can also just ask the data to `.describe()` itself...

In [8]:
# find out more info about your dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3190040 entries, 0 to 3190039
Data columns (total 18 columns):
year         category
serial       int32
hhwt         int16
statefip     category
countyfip    int16
gq           category
cinethh      category
cihispeed    category
pernum       int8
perwt        int16
relate       category
related      category
sex          category
age          category
race         category
raced        category
hispan       category
hispand      category
dtypes: category(13), int16(3), int32(1), int8(1)
memory usage: 100.4 MB


In [9]:
# describing your data
data.describe()

,serial,hhwt,countyfip,pernum,perwt
count,3.190040e+06,3.190040e+06,3.190040e+06,3.190040e+06,3.190040e+06
mean,6.918400e+05,9.614892e+01,5.099185e+01,2.083905e+00,1.021050e+02
std,4.064117e+05,7.594648e+01,8.809394e+01,1.340533e+00,8.314945e+01
min,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,3.350008e+05,5.100000e+01,0.000000e+00,1.000000e+00,5.300000e+01
50%,6.925320e+05,7.600000e+01,1.900000e+01,2.000000e+00,8.000000e+01
75%,1.047493e+06,1.170000e+02,7.300000e+01,3.000000e+00,1.240000e+02
max,1.394399e+06,2.401000e+03,8.100000e+02,2.000000e+01,2.401000e+03


Check out the `shape` of your data with it's `.shape` attribute. Notice the lack of parentheses.

In [10]:
data.shape

(3190040, 18)

***

#### Step 4: Trim your data

Right now you're working with your **masterfile** - a dataset containing everything you _could_ need for your analysis. You don't really want to modify this dataset because you might be using it for other analyses. For example, we're going to be analyzing access to high-speed internet in a state of your choosing but next week you might want to run the same analysis on another state or maybe just on a specific county. To make sure you can **reuse** your data and code later let's create an _analytical file_ or a _working dataset_, a dataset that contains only the data needed for **this** specific analysis at hand.

First, we are only interested in finding the _"Digital Divide"_ of one state right now. The **masterfile** contains data for all 50 states and the Disctric of Columbia. 

What you want to do is find all the rows where the `statefip` matches the your state's name. This is called boolean indexing.

Try the following
```python
data['statefip'] == 'ohio'
```
_Note: you can change 'ohio' to any other of the 50 states or 'district of columbia' for DC._

In [11]:
# try boolean indexing
data['statefip'] == 'ohio'

0          False
1          False
2          False
3          False
4          False
5          False
6          False
7          False
8          False
9          False
10         False
11         False
12         False
13         False
14         False
15         False
16         False
17         False
18         False
19         False
20         False
21         False
22         False
23         False
24         False
25         False
26         False
27         False
28         False
29         False
           ...  
3190010    False
3190011    False
3190012    False
3190013    False
3190014    False
3190015    False
3190016    False
3190017    False
3190018    False
3190019    False
3190020    False
3190021    False
3190022    False
3190023    False
3190024    False
3190025    False
3190026    False
3190027    False
3190028    False
3190029    False
3190030    False
3190031    False
3190032    False
3190033    False
3190034    False
3190035    False
3190036    False
3190037    Fal

This is going to return a `pandas.Series` of booleans (Trues and Falses) which then you can use to filter out any unnecessary rows.

It's good practice to save these as a variable early in your code (if you know them beforehand) or right before you use them in case you use these conditionals in more than one place. This is going to save you time if you decide to change the value you're comparing, `'ohio'` for `'california'` for example.

```python
mask_state = (data['statefip'] == 'ohio')
data[mask_state].head()
```

In [12]:
# try it yourself
mask_state = (data['statefip'] == 'ohio')
data[mask_state].head()

,year,serial,hhwt,statefip,countyfip,gq,cinethh,cihispeed,pernum,perwt,relate,related,sex,age,race,raced,hispan,hispand
2185099,2017,953654,55,ohio,0,households under 1970 definition,"no internet access at this house, apartment, o...",n/a (gq),1,55,head/householder,head/householder,male,79,white,white,not hispanic,not hispanic
2185100,2017,953654,55,ohio,0,households under 1970 definition,"no internet access at this house, apartment, o...",n/a (gq),2,69,spouse,spouse,female,79,white,white,not hispanic,not hispanic
2185101,2017,953655,82,ohio,49,households under 1970 definition,"yes, without a subscription to an internet ser...",n/a (gq),1,83,head/householder,head/householder,male,28,three or more major races,"white, black, aian",not hispanic,not hispanic
2185102,2017,953656,160,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",1,159,head/householder,head/householder,female,78,white,white,not hispanic,not hispanic
2185103,2017,953656,160,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",2,169,child,child,female,52,white,white,not hispanic,not hispanic


let's save it to another variable with a more useful name:

```python
state_data = data[mask_state].copy()
```

You have to use `.copy()` to create actual copies of the data. If you ran
```python
state_data = data[mask_state]
```
`state_data` would be a _view_ of the `data` dataframe. This can have unintended consequences down the road if you modify your dataframes. A lot of the times you'd get just a warning and your code will run just as intented - but why take risks, right?

In [13]:
# save your data to state_data
state_data = data[mask_state].copy()

Now, let's see what `.columns` we have in our dataframe. You can find these the same way you found the `.shape` of it earlier.

In [14]:
state_data.columns

Index(['year', 'serial', 'hhwt', 'statefip', 'countyfip', 'gq', 'cinethh',
       'cihispeed', 'pernum', 'perwt', 'relate', 'related', 'sex', 'age',
       'race', 'raced', 'hispan', 'hispand'],
      dtype='object')

Are there any columns that you are **confident** you don't need? If you are not 90% sure you won't need a variable don't drop it. 

Dropping columns is as easy using `.drop()` on your dataframe.

```python
state_data.drop(columns = ['list', 'of', 'columns', 'to', 'drop'])
```

In [15]:
state_data.drop(columns = ['related', 'raced', 'hispand'])

,year,serial,hhwt,statefip,countyfip,gq,cinethh,cihispeed,pernum,perwt,relate,sex,age,race,hispan
2185099,2017,953654,55,ohio,0,households under 1970 definition,"no internet access at this house, apartment, o...",n/a (gq),1,55,head/householder,male,79,white,not hispanic
2185100,2017,953654,55,ohio,0,households under 1970 definition,"no internet access at this house, apartment, o...",n/a (gq),2,69,spouse,female,79,white,not hispanic
2185101,2017,953655,82,ohio,49,households under 1970 definition,"yes, without a subscription to an internet ser...",n/a (gq),1,83,head/householder,male,28,three or more major races,not hispanic
2185102,2017,953656,160,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",1,159,head/householder,female,78,white,not hispanic
2185103,2017,953656,160,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",2,169,child,female,52,white,not hispanic
2185104,2017,953656,160,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",3,190,grandchild,male,26,white,not hispanic
2185105,2017,953656,160,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",4,174,other non-relatives,male,54,white,not hispanic
2185106,2017,953657,24,ohio,45,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",1,24,head/householder,female,54,white,not hispanic
2185107,2017,953657,24,ohio,45,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",2,21,spouse,male,55,white,not hispanic
2185108,2017,953657,24,ohio,45,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",3,31,child,female,28,white,not hispanic


If there are variables you _think_ you won't need but you're not very sure that's the case, you should explore them. 

`pandas` dataframe's columns are `pandas.Series` and they have methods and attributes just like dataframes.

Let's explore the variable `gq` which stands for `Group Quarters`. From the IPUMS [docs](https://usa.ipums.org/usa-action/variables/GQ#description_section):
>Group quarters are largely institutions and other group living arrangements, such as rooming houses and military barracks.

Let's see what `.unique()` values the `state_data['gq']` series has...

In [16]:
state_data['gq'].unique()

[households under 1970 definition, additional households under 1990 definition, additional households under 2000 definition, other group quarters, group quarters--institutions]
Categories (5, object): [households under 1970 definition < additional households under 1990 definition < group quarters--institutions < other group quarters < additional households under 2000 definition]

We can also see the `.value_counts()` which would give us a better idea of how useful this column might be. <br>For example, if a column has 2 values but 99% of the observations have one value and 1% have the other - you could drop column altogether since it might not add a lot value to your analysis. 

_Some variables have 100% of it's rows with the same value... \*cough\* \*cough\*_ `state_data['year']`...

In [17]:
state_data['gq'].value_counts()

households under 1970 definition               112619
other group quarters                             3032
group quarters--institutions                     3022
additional households under 1990 definition       149
additional households under 2000 definition        20
Name: gq, dtype: int64

From IPUMS [docs](https://usa.ipums.org/usa-action/variables/GQ#comparability_section):
>There are three slightly different definitions of group quarters in the IPUMS. For the period 1940-1970 (excluding the 1940 100% dataset), group quarters are housing units with five or more individuals unrelated to the householder. Before 1940 and in 1980-1990, units with 10 or more individuals unrelated to the householder are considered group quarters. **In the 2000 census, 2010 census, the ACS and the PRCS, no threshold was applied; for a household to be considered group quarters, it had to be on a list of group quarters that is continuously maintained by the Census Bureau. In earlier years, a similar list was used, with the unrelated-persons rule imposed as a safeguard.**

Because of this and the fact that most of our observations fall into the 1970 and 1990 definition, we'll stick to those 2 for our analysis.

Let's create another _mask_ to filter out households that don't fit our definition.

For multiple conditions we use: `&` and `|` operators (**and** and **or**, respectively)

In [18]:
mask_household = ( state_data['gq'] == 'households under 1970 definition' ) | ( state_data['gq'] == 'additional households under 1990 definition' )

**note**: another value added from having categorical variables is that, **if** they are ordered, you can use the `<`, `>` operators for conditions as well.
```python
mask_household = (state_data['gq'] <= 'additional households under 1990 definition')
```

In [19]:
state_data = state_data[mask_household].copy()

At this point you're really close to a `working_data` dataset. You have:
1. Kept one state's information and dropped the rest.
2. Kept only those _households_ you're interested in and dropped the rest

Our research question 1 is: "What share of households in X state have access to high-speed internet?"

Mathematically, 
$$ \frac{households\ with\ high\ speed\ internet}{households\ in\ state}$$

Your `state_data` dataset contains all you need to find the answer. 

***

#### Step 5: Save your data

Now that you have trimmed your **masterfile** into a `working_data` dataset you should save it. 

We've been working with a `.dta` file and it'd be best if we keep it that way. 

Try the following:
```python
state_data.to_stata(INTERIM_DATA_PATH / f'state_data-{today}.dta', write_index = False)
```

A few things:
1. We're using `f-strings` to tag our datafile with today's date.
2. You're turning off the `write_index` flag so you don't add a 'index' column to your `.dta` file. In this dataset, our index isn't meaningful. In other analysis you might have a meaningful index and you won't want to turn off this flag.

In [20]:
state_data.to_stata(INTERIM_DATA_PATH / f'state_data-{today}.dta', write_index = False)

***

#### Step 6: Bonus
What if we changed our research question a little bit, from <br>_"What share of households in X state have access to high-speed internet?_ <br>to <br>_"What share of households **with school-age children** in X state have access to high-speed internet?"_

This would be an interesting statistic to policy-makers, especially if we find discrepancies across demographic groups (research question 2).

The challenge here is that the **unit of observation** in our `state_data` file is a (weighted) person and we want to _filter_ out those **households** without any school-age children in them. This might sound a little complicated at first but it just requires modifying our previous workflow just a little.

We need to do a few things:
1. Define what we mean by school-age children.
2. Create a _mask_ to grab all households where these children are.
3. Create a list of unduplicated household identifiers (`'serial'`) 
4. Use that list to drop unwanted observations.

#### Step 6.1: School-age children

Most people would agree school age (Elementary through High School) is 6 - 17 year olds. Some people are interested in K-12 (5 - 17 or 18). Some people wouldn't include 18 year olds. Whatever measure you choose you must be able to defend why you are choosing it. 

For this analysis, I'll suggest we use 5 - 18 year olds (K-12) but you can choose whatever age range you want. Maybe high-school kids 14-18? That'd be interesting, you probably need access to high-speed internet at home a lot more in high school than you do in kindergarden. 

In [21]:
mask_children = (state_data['age'] >= '5') & (state_data['age'] <= '18')

<summary> <i>What data type is</i> <span style='font-family:monospace'>state_data['age']</span> <i>again?</i> 
    <details> 
        Categorical. This means that you even though its values _look_ like numbers, they're actually _value labels_ aka strings.
    </details>
</summary>

Now that we have our _mask_, we can use it to create a list of households with children in them.

Earlier we applied a mask to a dataframe and saved it to another variable. Here, we'll go a step further and grab just a column of that _filtered out_ dataframe.

Try it yourself first.

*Hint: How did we grab and explore a single column of a dataframe earlier?*

In [22]:
households_with_children = state_data[mask_children]['serial'].copy()

In [23]:
households_with_children.head()

2185119    953662
2185129    953668
2185137    953671
2185138    953671
2185165    953685
Name: serial, dtype: int32

How do you think we can `.drop_duplicates()`?

In [24]:
households_with_children = households_with_children.drop_duplicates()

Once you have your unduplicated list of households with children all you have to do is to check if a `serial` value from our `state_data` dataset `.isin()` our `households_with_children` series.

In [26]:
mask_households_with_children = state_data['serial'].isin(households_with_children)
state_data[mask_households_with_children].head()

,year,serial,hhwt,statefip,countyfip,gq,cinethh,cihispeed,pernum,perwt,relate,related,sex,age,race,raced,hispan,hispand
2185117,2017,953662,57,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",1,58,head/householder,head/householder,female,48,white,white,not hispanic,not hispanic
2185118,2017,953662,57,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",2,62,child,child,male,20,white,white,not hispanic,not hispanic
2185119,2017,953662,57,ohio,0,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",3,78,child,child,female,9,white,white,not hispanic,not hispanic
2185128,2017,953668,140,ohio,61,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",1,140,head/householder,head/householder,male,28,black/african american/negro,black/african american/negro,not hispanic,not hispanic
2185129,2017,953668,140,ohio,61,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",2,192,sibling,sibling,female,16,black/african american/negro,black/african american/negro,not hispanic,not hispanic


Let's save that as our `working_data` dataset and save that to memory.

In [27]:
working_data = state_data[mask_households_with_children].copy()

```python
working_data.to_stata(INTERIM_DATA_PATH / f'working_data-{today}.dta', write_index = False)
```

In [28]:
working_data.to_stata(INTERIM_DATA_PATH / f'working_data-{today}.dta', write_index = False)